## Get Best-Performing Hyper Parameters

In [1]:
!which python

/Users/SXH1M01/anaconda3/envs/phd_py36/bin/python


In [2]:
!pip freeze

appnope==0.1.0
asn1crypto==0.24.0
attrs==19.1.0
backcall==0.1.0
bleach==3.1.0
boto==2.47.0
boto3==1.9.184
botocore==1.12.184
certifi==2019.6.16
cffi==1.12.3
chardet==3.0.4
costcla==0.5
cryptography==2.7
cycler==0.10.0
cymem==2.0.2
cytoolz==0.9.0.1
decorator==4.4.0
defusedxml==0.6.0
dill==0.2.9
docutils==0.14
entrypoints==0.3
gensim==0.13.4
h5py==2.7.0
html5lib==0.999
idna==2.8
ipykernel==5.1.1
ipython==7.6.1
ipython-genutils==0.2.0
ipywidgets==7.5.0
jedi==0.14.0
Jinja2==2.10.1
jmespath==0.9.4
joblib==0.9.4
json5==0.8.5
jsonschema==3.0.1
jupyter==1.0.0
jupyter-client==5.3.0
jupyter-console==6.0.0
jupyter-core==4.5.0
jupyterlab==1.0.1
jupyterlab-server==1.0.0
MarkupSafe==1.1.1
matplotlib==2.0.0
mistune==0.8.4
mkl-fft==1.0.6
mkl-random==1.0.1
msgpack==0.6.1
msgpack-numpy==0.4.3.2
murmurhash==1.0.2
nb-conda==2.2.1
nb-conda-kernels==2.2.2
nbconvert==5.5.0
nbformat==4.4.0
nltk==3.2.2
nose==1.3.7
notebook==5.7.8
numpy==1.16.4
pandas==0.19.2
pandocfilters==1.4.2
parso==0.5.0
pexpect==4.7.0
pic

In [3]:
import pandas as pd
import pymongo

In [4]:
client = pymongo.MongoClient(serverSelectionTimeoutMS=100, host="127.0.0.1")
db = client.metrics_causal_model_reranker

In [5]:
import datetime

def fmt_dt(date_time_str):
    dt= datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
    return dt.strftime("%m-%d-%Y %H:%M")

def query_collection(collection):
    db = client.metrics_causal_model_reranker
    if "SENT_" in collection:
        db = client.metrics_causal_model_parser
        
    project = {
            "params": "$parameters",
            "micro_f1": "$MICRO_F1",
            "asof": "$asof",
            "_id": 1
        }
    feats_pipeline = [{ "$project": project }]
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    if len(rows) == 0:
        return pd.DataFrame([])

    results = []
    for r in rows:
        d = dict(r["params"])
        d.update(r["micro_f1"])
        d["asof"] = str(r["asof"])
        results.append(d)
    
    df = pd.DataFrame(results)
    df["extractors"] = df["extractors"].apply(lambda l: ",".join(l))
#     df["asof"] = df["asof"].apply(fmt_dt)
    df = df.sort_values(by="f1_score",ascending=False)
    return df

In [6]:
def get_df(collection):
    assert "_VD" in collection
    cols = ["best_top_n", "C", "best_max_parses", 
        #"best_max_upd", 
        "max_update_items", 
        # "best_min_prob", "extractors", 
        "initial_weight", "loss_type",\
        #"min_feat_freq",
        "pa_type", "early_stopping_iters",]    
    
    if "SENT" in collection:
        cols = []
    elif "PCPTRN" in collection:
        cols = ["best_top_n", "learning_rate", "best_max_parses", 
        #"best_max_upd", 
        "max_update_items", 
        # "best_min_prob", "extractors", 
        "initial_weight","early_stopping_iters",]
        #"min_feat_freq"]
    
    # add common cols
    cols = ["f1_score", "precision", "recall", "asof"] + cols + ["extractors", "num_feats_MEAN"]
    df = query_collection(collection)
    return df[cols]

In [7]:
col = "SC_STR_PCPTRN_RE-RANKER_HYPER_PARAM_VD"
get_df(col).head(1)

,f1_score,precision,recall,asof,best_top_n,learning_rate,best_max_parses,max_update_items,initial_weight,early_stopping_iters,extractors,num_feats_MEAN
47,0.810458,0.854811,0.770479,2019-07-06 08:14:46.773000,10,1.0,300,1,0.01,1,"Above-,Inv-,num_crels,CChainStats-",88.6


In [8]:
# MONGO_COLLECTION = "SC_RE-RANKER_HYPER_PARAM_VD"
# MONGO_COLLECTION = "SC_COST_INSENS_RE-RANKER_HYPER_PARAM_VD"

## CB

### VD

In [9]:
df_pa = get_df("SENT_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD")
df_pa.head(1)

,f1_score,precision,recall,asof,extractors,num_feats_MEAN
0,0.74358,0.758542,0.729197,2019-05-03 14:18:47.905000,"single_words,between_word_features,label_set,t...",27479.6


In [10]:
# df_pa = query_collection("CB_STR_PCPTRN_RE-RANKER_FEATURE_SEL_VD")[["f1_score", "early_stopping_iters", "extractors"]]
# df_pa.head(1)

In [11]:
df_pa = get_df("CB_STR_PCPTRN_RE-RANKER_HYPER_PARAM_VD")
df_pa.head(1)

,f1_score,precision,recall,asof,best_top_n,learning_rate,best_max_parses,max_update_items,initial_weight,early_stopping_iters,extractors,num_feats_MEAN
29,0.741353,0.782873,0.704015,2019-06-24 20:43:02.777000,1,0.1,300,1,0.01,2,"Prob-,Above-",33.0


In [13]:
query_collection("CB_PA_RE-RANKER_HYPER_PARAM_VD")

""


In [12]:
df_pa = get_df("CB_PA_RE-RANKER_HYPER_PARAM_VD")
df_pa.head(1)

KeyError: "['f1_score' 'precision' 'recall' 'asof' 'best_top_n' 'C' 'best_max_parses'\n 'max_update_items' 'initial_weight' 'loss_type' 'pa_type'\n 'early_stopping_iters' 'extractors' 'num_feats_MEAN'] not in index"

In [ ]:
df_pa = get_df("CB_RE-RANKER_HYPER_PARAM_VD")
df_pa.head(1)

### Test

In [ ]:
df_pa = get_df("SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD")
df_pa.head(10)

In [ ]:
df_pa = get_df("TEST_CB_STR_PCPTRN_RE-RANKER_VD")
df_pa.head(10) # Improved precision over recall. Top n was restricted to 1, so only capable of upping pr

In [ ]:
df_pa = get_df("TEST_CB_PA_RE-RANKER_VD")
df_pa.head(10)

In [ ]:
# To improve upon this, you can use early stopping
df_pa = get_df("TEST_CB_RE-RANKER_VD")
df_pa.head(10)

## SC

### VD

In [ ]:
df_pa = get_df("SENT_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD")
df_pa.head(1)

In [ ]:
# cols = ["f1_score", "extractors", "early_stopping_iters"]
# query_collection("SC_STR_PCPTRN_RE-RANKER_FEATURE_SEL_VD_2").head(1)[cols]

In [ ]:
df_pa = get_df("SC_STR_PCPTRN_RE-RANKER_HYPER_PARAM_VD")
df_pa.head(1)

In [ ]:
df_pa = get_df("SC_PA_RE-RANKER_HYPER_PARAM_VD")
df_pa.head(1)

In [ ]:
df_pa = get_df("SC_RE-RANKER_HYPER_PARAM_VD")
df_pa.head(1)

### Test

In [ ]:
df_pa = get_df("SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD")
df_pa.head(10)

In [ ]:
df_pa = get_df("TEST_SC_STR_PCPTRN_RE-RANKER_VD")
df_pa.head(10)

In [ ]:
df_pa = get_df("TEST_SC_PA_RE-RANKER_VD")
df_pa.head(10)

In [ ]:
df_pa = get_df("TEST_SC_RE-RANKER_VD")
df_pa.head(10)

## Feature Selection

In [ ]:
df_pa = query_collection("CB_STR_PCPTRN_RE-RANKER_FEATURE_SEL_VD")[["f1_score","learning_rate","max_update_items", "extractors"]]
df_pa.head(1)

In [ ]:
df_pa = query_collection("CB_STR_PCPTRN_RE-RANKER_FEATURE_SEL_VD")[["f1_score","learning_rate","max_update_items", "extractors"]]
df_pa.head(1)

In [ ]:
df_pa = query_collection("SC_STR_PCPTRN_RE-RANKER_FEATURE_SEL_VD")[["f1_score","learning_rate","max_update_items"]]
df_pa.head(1)